## Group Level Analysis

This notebook is designed to perform a voxelwise analysis of the infant CBF data

In [1]:
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.fsl.utils import Merge
from nipype.interfaces.fsl.model import Randomise
from nipype.interfaces.fsl.maths import ApplyMask

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

#other study-specific variables
#project_home = '/Volumes/iang/active/BABIES/BABIES_perfusion'
project_home = '/Users/catcamacho/Box/SNAP/BABIES'
output_dir = project_home + '/proc/asl_group'
preproc_dir = project_home + '/proc/asl_preproc'
wkflow_dir = project_home + '/workflows'
template = project_home + '/templates/T2w_BABIES_template_2mm.nii.gz'
mask = project_home + '/templates/BABIES_gm_mask_2mm.nii.gz'

# Files for group level analysis
group_mat = project_home + '/misc/design_kmg.mat'
t_contrasts = project_home + '/misc/tcon_kmg.con'

In [2]:
# Data Handling Nodes

datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir

grabcbfdata = Node(DataGrabber(template=preproc_dir + '/cbf_volume/*/salsop_cbf_masked.nii.gz', 
                               sort_filelist=True, 
                               outfields=['cbf_list']), 
                   name='grabcbf')


In [3]:
# Analysis Nodes

merge = Node(Merge(dimension = 't'), name = 'merge')

apply_mask = Node(ApplyMask(mask_file=mask, nan2zeros=True), name='apply_mask')

randomise = Node(Randomise(tfce = False,
                           num_perm = 500,
                           tcon = t_contrasts,
                           demean = True,
                           design_mat = group_mat), name = 'randomise')

In [5]:
# Analysis workflow

grouplevel = Workflow(name='grouplevel')

grouplevel.connect([(grabcbfdata, merge,[('cbf_list', 'in_files')]),
                    (merge, apply_mask, [('merged_file','in_file')]),
                    (apply_mask, randomise, [('out_file', 'in_file')]),
                    (randomise, datasink, [('t_corrected_p_files', 'kmg_t_corrected_p_files_cesd')]),
                    (randomise, datasink, [('tstat_files', 'kmg_tstat_files_cesd')])
                   ])

grouplevel.base_dir = wkflow_dir
grouplevel.write_graph(graph2use='flat')
grouplevel.run('MultiProc', plugin_args={'n_procs': 2})


180702-20:27:11,591 workflow INFO:
	 Generated workflow graph: /Users/catcamacho/Box/SNAP/BABIES/workflows/grouplevel/graph.png (graph2use=flat, simple_form=True).
180702-20:27:11,617 workflow INFO:
	 Workflow grouplevel settings: ['check', 'execution', 'logging', 'monitoring']
180702-20:27:11,629 workflow INFO:
	 Running in parallel.
180702-20:27:11,633 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 14.40/14.40, Free processors: 2/2.
180702-20:27:11,683 workflow INFO:
	 [Node] Setting-up "grouplevel.grabcbf" in "/Users/catcamacho/Box/SNAP/BABIES/workflows/grouplevel/grabcbf".
180702-20:27:11,699 workflow INFO:
	 [Node] Running "grabcbf" ("nipype.interfaces.io.DataGrabber")
180702-20:27:11,720 workflow INFO:
	 [Node] Finished "grouplevel.grabcbf".
180702-20:27:13,637 workflow INFO:
	 [Job 0] Completed (grouplevel.grabcbf).
180702-20:27:13,639 workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 14.40/14.40, Free processors